In [3]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# RQ1: Which technologies have been investigated in the last decade?

In [22]:
#Datene einlesen
#data = pd.read_csv('./data/data.csv')
#data.drop(["Unnamed: 0"], axis = 1, inplace = True)

data = pd.read_csv('./data/export1.csv',dtype={'Abstract': str}, usecols = ['Authors','Year'])
data.head()

,Authors,Abstract
0,T. Dingsøyr; F. O. Bjørnson; F. Shull,There have been many claims about knowledge ma...
1,G. Li; H. Dong; Q. Zheng; M. Zhou; Y. Guo,The article analyzed the basic life cycle mode...
2,M. Ardis; P. Bourque; T. Hilburn; K. Lasfer; S...,The paper mentions that a reference curriculum...
3,O. Hazzan,Although people-related issues are central fac...
4,P. Johnson; M. Ekstedt; I. Jacobson,"Darwin's theory of natural selection, Maxwell'..."


,Document Title,Publication Year,Abstract
0,What Do We Know about Knowledge Management? Pr...,2009,There have been many claims about knowledge ma...
1,Research on National and International Softwar...,2009,The article analyzed the basic life cycle mode...
2,Advancing Software Engineering Professional Ed...,2011,The paper mentions that a reference curriculum...
3,Putting Human Aspects of Software Engineering ...,2010,Although people-related issues are central fac...
4,Where's the Theory for Software Engineering?,2012,"Darwin's theory of natural selection, Maxwell'..."
...,...,...,...
1941,Agility and Architecture: Can They Coexist?,2010,Agile development has significantly impacted i...
1942,Front Cover,2011,September/October IEEE Software: Engineering F...
1943,Table of Contents,2009,November/December 2009 IEEE Software Table of ...
1944,Table of Contents,2009,IEEE Software September/October 2009 issue on ...


In [5]:
data.columns

Index(['Document Title', 'Publication Year', 'Abstract'], dtype='object')

In [6]:
data.count()

Document Title      1946
Publication Year    1946
Abstract            1933
dtype: int64

In [28]:
def CreateTermDocMatrix(column):
        
    count_vectorizer = CountVectorizer(stop_words='english',ngram_range = (1,2))
    sparse_matrix = count_vectorizer.fit_transform(data[column].values.astype('U'))

    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names())
    df['year'] = data['year']
    returndf = df.groupby('year').sum().transpose()
    return returndf

In [29]:
def ShowWord(data,word):
    d = data.loc[word]
    d.plot.bar()

In [30]:
def SpaltenEntf(data):
    y = []
    for item in data.columns:
        y = data.columns


    for index in range(len(y)):
        x = y[index]
        if x in stopwords or x.isnumeric():
            del data[y[index]]
    return data

In [31]:
#Untersuchung Abstract
#abstract = CreateTermDocMatrix('Abstract')

In [32]:
#b = abstract.loc[abstract[2011] > 20] 

In [33]:
#b.plot.bar()
#print(b)

In [34]:
from collections import Counter
#cnt = Counter()

#for abstract in data['Abstract']:
#    if type(abstract) is str:
#        wordlist = abstract.split(' ')
#        for word in wordlist:
#            cnt[word] += 1
        
#cnt.most_common(30)

In [35]:
#for abstract in data.itertuples():
#    print(abstract[1])
#    print(abstract[2])
#    print(abstract[3])

In [17]:
data = pd.read_csv('./data/data.csv',usecols = ['Document Title','Abstract','Publication Year'])
data.columns = ['title','abstract','year']

In [18]:
#Untersuchung Title
title = CreateTermDocMatrix('abstract')

#print(title)
#title = SpaltenEntf(title)
#print(title)

#b = title.loc[title[2017] > 5] 

#b.plot.bar()
#print(b)
#b.shape
title

NameError: name 'CreateTermDocMatrix' is not defined

In [16]:
#Einzelnes Wort anzeigen
#abstract = CreateTermDocMatrix('Abstract')
#ShowWord(abstract,'software')

In [21]:
import spacy
from spacy.symbols import NOUN
from collections import Counter


nlp = spacy.load("en_core_web_sm")

cnt = Counter()
tokenlist = []

for columns in data.itertuples():

    abstract = columns[1] #1 = title 2 = year 3 = abstract
    #print(type(abstract))
    # Verarbeite den Text
    if type(abstract) is str:
        doc = nlp(abstract)
        
        for token in doc:
            # Greife auf den Text, die Wortart und die Dependenzrelation des Tokens zu
            token_text = token.text
            token_pos = token.pos_
            token_dep = token.dep_
            token_ent = token.ent_type_
            next_token = doc[token.i + 1]
            next_token1 = doc[token.i + 2]
            next_token2 = doc[token.i + 3]
            pref_token = doc[token.i - 1]
            if next_token.text == ";":
                print("Author", pref_token.text,token.text)
                print(next_token1, next_token2)


                # Dies dient nur zur Formatierung
        #        print(f"{token_text:<12}{token_pos:<10}{token_dep:<10}")
 
#cnt.most_common(50)
#tokenlist

IndexError: [E040] Attempt to access token at 13, max length 13.

In [8]:
data

,Authors,Abstract
0,T. Dingsøyr; F. O. Bjørnson; F. Shull,There have been many claims about knowledge ma...
1,G. Li; H. Dong; Q. Zheng; M. Zhou; Y. Guo,The article analyzed the basic life cycle mode...
2,M. Ardis; P. Bourque; T. Hilburn; K. Lasfer; S...,The paper mentions that a reference curriculum...
3,O. Hazzan,Although people-related issues are central fac...
4,P. Johnson; M. Ekstedt; I. Jacobson,"Darwin's theory of natural selection, Maxwell'..."
...,...,...
1941,P. Abrahamsson; M. A. Babar; P. Kruchten,Agile development has significantly impacted i...
1942,NaN,September/October IEEE Software: Engineering F...
1943,NaN,November/December 2009 IEEE Software Table of ...
1944,NaN,IEEE Software September/October 2009 issue on ...
